In [1]:
## De functie die tf-records leest en ze uiteindelijk weer wegschrijft (maar dan met minder labels) heet readTfRecord
## Deze functie roept andere functies aan: getLabels en getLabelIndices

import numpy as np
import pandas as pd
import re
import os
import random
from collections import Counter

import tensorflow as tf
from tensorflow import gfile

In [2]:
location = "hugo" # "urban" "amsterdam" "jungle" 
# if we make a new location / or project we need to do the following:
# create a csv labels file
# create a csv labels prop file
# create the new directories that hold the new train and eval samples
#
# for example the new project `hugo`
# in ..serval/dataset/classlabels/csv_files we need 
# class_labels_indices_hugo.csv
# class_labels_indices_hugo_prop.csv (copy of class_labels_indices_hugo.csv)
# in this csv we need to add a column `target_ind` and put all zeros, this is a placeholder of the propertions per class
# train_hugo_new and eval_hugo_new in serval/dataset


In [3]:
# inside docker goto right place 
os.chdir("/tf/serval/serval")
# set relvant full paths
# /tf is the tensorflow jupyter container mapping
path_original_tf_records = "dataset/audioset_v1_embeddings/"
path_new_tf_records = "dataset/"
path_class_labels = "dataset/classlabels/csv_files/"
#os.getcwd()

In [4]:
## Paden naar originele tf-records: bal_train, unbal_train en eval
bal_train_data_pattern = path_original_tf_records + "bal_train/*.tfrecord"
unbal_train_data_pattern = path_original_tf_records + "unbal_train/*.tfrecord"
eval_data_pattern = path_original_tf_records + "eval/*.tfrecord"

## Paden voor nieuwe tf-records (train en test)
train_pattern_new = path_new_tf_records + "train_" + location + "_new"
eval_pattern_new = path_new_tf_records + "eval_" + location + "_new"

In [5]:
## Boolean die aangeeft of alle examples worden geschreven naar een tf-record
## Bij false: upsamplen/downsamplen adhv proporties
write_all_examples = True
## proportie train/test
pct_train = 0.6

In [6]:
## Lijst van alle labels (nieuwe) en alle combi's
all_labels = []
label_combinations = []

In [7]:
## Open csv-bestanden met class labels
## Oorspronkelijke csv-file google audioset class labels
mid_to_label_old = pd.read_csv(path_class_labels + "class_labels_indices_original_audioset.csv", sep=",")

## csv-file met labels (urban/jungle/amsterdam)
new_label_file = "class_labels_indices_" + location + ".csv"
mid_to_label_new = pd.read_csv(path_class_labels + new_label_file, sep=";")
## mid_to_label_new = pd.read_csv('csv_files/class_labels_indices_urban.csv', sep=";")

## Naam bestand om proporties naartoe te schrijven (of om te gebruiken bij sampelen)
## Als deze nog niet bestaat: maak een kopietje van mid_to_labels_new met de naam die hieronder staat
name_label_prop_file = "class_labels_indices_" + location + "_prop.csv"
label_proportions = pd.read_csv(path_class_labels + name_label_prop_file, sep=";")

In [8]:
mid_to_label_new

,index,mid,display_name
0,0,/t/dd00134,"Car, boat, motorcyle, truck"
1,1,/m/01j4z9,Chainsaw
2,2,/m/07pczhz,Chop
3,3,/m/032s66,"Gunshot, gunfire"
4,4,/m/09ct_,Helicopter
5,5,/t/dd00003,Male singing
6,6,/m/07rpkh9,Cattle
7,0,/m/02rlv9,"Car, boat, motorcyle, truck"
8,0,/m/04_sv,"Car, boat, motorcyle, truck"
9,7,/m/06bz3,Radio


In [9]:
## Deze functie checkt of een example wordt gemaakt (downsamplen)
def checkIfNewExample(labels):
    
    ## Alleen bij 1 label worden er examples overgeslagen (of bij write_all_examples)
    if(len(labels)>1 or write_all_examples):
        return True
    else:
        label = list(labels)[0]
        prop = label_proportions.loc[label_proportions['index']==np.int(label), "target_ind"].values[0]
        rand = random.random()
        ## Als random niet proportie overschrijdt, return true
        if rand <= prop:
            return True
        else:
            return False
    return False

In [10]:
## Deze functie checkt of er nog een example moet worden gemaakt (upsamplen)
## Momenteel kan het aantal enkel verdubbeld worden

def checkIfExtraExample(labels):
    
    ## Alleen bij 1 label worden er examples gekopieerd (of als er niet (up)gesampled wordt)
    if(len(labels)>1 or write_all_examples):
        return False
    else:
        label = list(labels)[0]
        prop = label_proportions.loc[label_proportions['index']==np.int(label), "target_ind"].values[0]-1
        rand = random.random()
        ## Als random niet proportie overschrijdt, return true
        if rand <= prop:    
            return True
        else:
            return False
    return False

In [11]:
#### Find new label for each of old label (527 -> x)

def getNewLabel(label_old):
    
    ## Get m-id that matches with index
    mid = mid_to_label_old.loc[mid_to_label_old['index']==label_old, 'mid']

    ## Find the new label belonging to this m-id
    label_new = mid_to_label_new.loc[mid_to_label_new['mid']==str(mid.values[0]), 'index']
    
    return label_new

In [12]:
def createExample(example_old, new_labels):
    
    ## Bouw structuur van tf-record met video-id, labels en features
    
    audio_embedding = example_old.feature_lists.feature_list['audio_embedding']
    feature_lists = tf.train.FeatureLists(feature_list={"audio_embedding": audio_embedding})
    #print("New labels: " + str(new_labels))
    
    label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=new_labels))
    video_id = example_old.context.feature['video_id'].bytes_list.value
    videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id)) 

    context_feats = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat})

    sequence_example = tf.train.SequenceExample(context=context_feats, feature_lists=feature_lists)
    
    return sequence_example

In [13]:
## Deze code haalt voor een label alle (grand)parents op
## Momenteel worden deze functies niet meer gebruikt.
## Maar schroom je niet om eens te kijken hoe deze prachtige recursieve functie werkt!

def getParents(label, parents):
    
    parent = int(mid_to_label_new.loc[mid_to_label_new['index']==label, 'parent'])
    
    if parent==999:
        return parents
    else:
        parents.append(parent)
        return getParents(parent, parents)
    return parent

def getLabelsWithParents(labels):
    total_labels = labels
    for label in labels:
        parents = getParents(label, [])
        total_labels.extend(parents)
    return(set(total_labels))

In [14]:
#### Lees tf-records en iterate over de examples#### Lee 
#### Voor elk example wordt de video-id gepakt en de bijbehorende lijst m-id's gezocht (in balanced_segment.csv)

## 1 tf-record heeft meerdere examples
## Deze functie checkt voor elk example of het een label heeft uit onze labelset

## Input: filename van tf-record (pad) en prefix (vanwege dubbele filenames)
## Result: schrijven van nieuw tf-record

## Geef prefix mee (om dubbele namen te voorkomen)
def readWriteTfRecord(tfrecords_filename, prefix):
    
    ## Maak 2 writers (train en test) die nieuw tf-records schrijft
    
    # windows 
    #record_name = tfrecords_filename.split('\\')[-1]
    # linux
    record_name = tfrecords_filename.split('/')[-1]
    ## Let op: hij doet moeilijk over AUX.tfrecord. Dus prefix A bij UX mag niet
    #if(str(prefix+record_name_=="AUX.tfrecord"):
    #    record_name="UXx.tfrecord"
        
    ## Nieuwe filenames: 1 voor train en 1 voor eval
    train_file_new = str(train_pattern_new + '/' + prefix + record_name)
    eval_file_new = str(eval_pattern_new + '/' + prefix + record_name)

    train_writer = tf.python_io.TFRecordWriter(train_file_new)
    eval_writer = tf.python_io.TFRecordWriter(eval_file_new)
    nr_new_examples = 0
    
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        ## Get labels and find for each label the m-code in class_label_indices_old
        labels = example.context.feature['labels'].int64_list.value
        
        new_labels = []
        for label in labels:
            ## Hij returnt een set van cellen (of leeg of van 1 rij)
            label_new = getNewLabel(label)
            
            if(label_new.shape[0]>0):
                new_labels.append(label_new.values[0])
        
        ## Als enkele (grand)parents ontbreken, voeg ze toe
        #new_labels = getLabelsWithParents(new_labels)
        ## If any of the labels left (for our model), write new tf-record
        
        if (len(new_labels) > 0 and checkIfNewExample(new_labels)):
        #if(len(new_labels) > 0):
            ## Add to all labels
            all_labels.extend(new_labels)
            label_combinations.append(new_labels)
   
            sequence_example = createExample(example, new_labels)
            nr_new_examples += 1
            
            ## Write the example
            rand = random.random()
            if rand <= pct_train:
                train_writer.write(sequence_example.SerializeToString())
            else:
                eval_writer.write(sequence_example.SerializeToString())
        
            ## Check of er nog een example moet worden gemaakt (upsamplen)
            if(checkIfExtraExample(new_labels)):
                rand = random.random()
                if rand <= pct_train:
                    train_writer.write(sequence_example.SerializeToString())
                else:
                    eval_writer.write(sequence_example.SerializeToString())
        
                all_labels.extend(new_labels)
                label_combinations.append(new_labels)
            print('Example gemaakt! Video-id: ' + str(example.context.feature['video_id'].bytes_list.value)
                  + ' labels : ' + str(new_labels))

    #print("nr new examples: " + str(nr_new_examples))
    
    # Only write if any examples created

    if nr_new_examples > 0:
        train_writer.close()
        eval_writer.close()
        #print("tfrecord written")
    else:
        #print("empty file.. remove: " + path_tfrecord_new)
        os.remove(train_file_new)
        os.remove(eval_file_new)

In [15]:
## Maak nieuwe tf-records van bal_train
# make sure the new directory location exists train_amsterdam_new
# ISSUE amsterdam has new labels? csv not ; separated

files = gfile.Glob(bal_train_data_pattern)

for file in files:
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'B')

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Example gemaakt! Video-id: [b'ic_xXsNZ9aM'] labels : [10]
Example gemaakt! Video-id: [b'3udWS-cMb-8'] labels : [3]
Example gemaakt! Video-id: [b'IR_ks3y_UTA'] labels : [3]
Example gemaakt! Video-id: [b'p0YtKLkJnug'] labels : [2]
Example gemaakt! Video-id: [b'-s9kwrRilOY'] labels : [0]
Example gemaakt! Video-id: [b'2-uMMdsFEWM'] labels : [0]
Example gemaakt! Video-id: [b'19DRe0cdeXQ'] labels : [9]
Example gemaakt! Video-id: [b'BYNisb8TQ9g'] labels : [4]
Example gemaakt! Video-id: [b'qogFRsecYSw'] labels : [0]
Example gemaakt! Video-id: [b'Ptxjrmqo2Xo'] labels : [1]
Example gemaakt! Video-id: [b'u-mrVxR5bNo'] labels : [6]
Example gemaakt! Video-id: [b'MLVnHZCswq4'] labels : [9]
Example gemaakt! Video-id: [b'grIP4c56AsM'] labels : [9]
Example gemaakt! Video-id: [b'Xug8OJNzEvI'] labels : [10]
Example gemaakt! Video-id: [b'SsB-AP-fcSY'] labels : [10]
Example gemaakt! Video-id: [b'6kJNnElIDs8'] labels : [5]


Example gemaakt! Video-id: [b'MsggONi5frI'] labels : [2]
Example gemaakt! Video-id: [b'GJvWWCx2UuU'] labels : [10]
Example gemaakt! Video-id: [b'GJd9qaGm92I'] labels : [3]
Example gemaakt! Video-id: [b'CxmMmFS5Arw'] labels : [1]
Example gemaakt! Video-id: [b'_q7bllBtC2A'] labels : [0]
Example gemaakt! Video-id: [b'mFwURBjeSn8'] labels : [1]
Example gemaakt! Video-id: [b'orlpRanqgAc'] labels : [6]
Example gemaakt! Video-id: [b'ICtHsbPozLU'] labels : [10]
Example gemaakt! Video-id: [b'ICdovrQhhrw'] labels : [10]
Example gemaakt! Video-id: [b'O2htSqXhdqE'] labels : [9]
Example gemaakt! Video-id: [b'lyuULnLsc4w'] labels : [4]
Example gemaakt! Video-id: [b'fX18D1VUEAI'] labels : [6]
Example gemaakt! Video-id: [b'7e2ifgqrN1Q'] labels : [0]
Example gemaakt! Video-id: [b'6qi1KV9MoO0'] labels : [10]
Example gemaakt! Video-id: [b'6qY1ojVpxB4'] labels : [5]
Example gemaakt! Video-id: [b'9ww0jrYLgZ8'] labels : [3]
Example gemaakt! Video-id: [b'tTtuoCraB1Y'] labels : [3]
Example gemaakt! Video-id: 

Example gemaakt! Video-id: [b'-rIyGq-gIQw'] labels : [4]
Example gemaakt! Video-id: [b'BTxwsd0wOMg'] labels : [1]
Example gemaakt! Video-id: [b'dlSyVxxbptE'] labels : [6]
Example gemaakt! Video-id: [b'LTaJhm98DzQ'] labels : [6]
Example gemaakt! Video-id: [b'opnG6qoVYzM'] labels : [3]
Example gemaakt! Video-id: [b'opXNUOAPTxA'] labels : [0]
Example gemaakt! Video-id: [b'AW3agv6wpyw'] labels : [2]
Example gemaakt! Video-id: [b'CnYWJp2bknU'] labels : [0]
Example gemaakt! Video-id: [b'S5ONWWKaMIA'] labels : [10]
Example gemaakt! Video-id: [b'jsM5T7ywvLQ'] labels : [6]
Example gemaakt! Video-id: [b'HA5MeuJwgw4'] labels : [1]
Example gemaakt! Video-id: [b'-fwZGpmJNhw'] labels : [10]
Example gemaakt! Video-id: [b'_WD9mbwAcrQ'] labels : [8]
Example gemaakt! Video-id: [b'oh08bbDVFZM'] labels : [9]
Example gemaakt! Video-id: [b'dJudErPaMWI'] labels : [10]
Example gemaakt! Video-id: [b'oQs4NFBTxd8'] labels : [9]
Example gemaakt! Video-id: [b'gVxVSH8_Z2U'] labels : [6]
Example gemaakt! Video-id: [

Example gemaakt! Video-id: [b'Rzpe6JTlarU'] labels : [6]
Example gemaakt! Video-id: [b'cjXFIp4UyY0'] labels : [0]
Example gemaakt! Video-id: [b'pZF-hff52ts'] labels : [3]
Example gemaakt! Video-id: [b'Q975D6bpK_c'] labels : [0]
Example gemaakt! Video-id: [b'sq3jcffMfwU'] labels : [3]
Example gemaakt! Video-id: [b'-JbPonixXS4'] labels : [1]
Example gemaakt! Video-id: [b'-JOKwS64mRU'] labels : [10]
Example gemaakt! Video-id: [b'-JuVh6islEI'] labels : [7]
Example gemaakt! Video-id: [b'JPSwpnLAdXc'] labels : [1]
Example gemaakt! Video-id: [b'qBLiT_0wwM4'] labels : [3]
Example gemaakt! Video-id: [b'wcGst9bsj1Q'] labels : [1]
Example gemaakt! Video-id: [b'OxvKEoC_cUU'] labels : [6]
Example gemaakt! Video-id: [b'bTC22WA2gJk'] labels : [8]
Example gemaakt! Video-id: [b'bT-CEaOEGdI'] labels : [0]
Example gemaakt! Video-id: [b'lRZcTLOAfdc'] labels : [6]
Example gemaakt! Video-id: [b'IhRkgDB1Suw'] labels : [2]
Example gemaakt! Video-id: [b'IhDfxIBqaLA'] labels : [8]
Example gemaakt! Video-id: [b'

Example gemaakt! Video-id: [b'A7v7zrZZEjA'] labels : [4]
Example gemaakt! Video-id: [b'PyxzFVgz70M'] labels : [3]
Example gemaakt! Video-id: [b'llgsi8uZDTc'] labels : [0]
Example gemaakt! Video-id: [b'llpnRcwTZ9Y'] labels : [8]
Example gemaakt! Video-id: [b'pu0Xo4gjA7c'] labels : [10]
Example gemaakt! Video-id: [b'pusDxENmp9Q'] labels : [0]
Example gemaakt! Video-id: [b'rTSz2mDpLoo'] labels : [0]
Example gemaakt! Video-id: [b'rTaWJ4gZ7g0'] labels : [3]
Example gemaakt! Video-id: [b'17bfDrV7bvY'] labels : [0]
Example gemaakt! Video-id: [b'EbH-AtIHmAM'] labels : [0]
Example gemaakt! Video-id: [b'QtTzb_VAu-k'] labels : [6]
Example gemaakt! Video-id: [b'q-P6YlGm4uY'] labels : [3]
Example gemaakt! Video-id: [b'_lGsMYx8ezI'] labels : [10]
Example gemaakt! Video-id: [b'TeA1puWNGgw'] labels : [10]
Example gemaakt! Video-id: [b'swzCAxFJaWo'] labels : [8]
Example gemaakt! Video-id: [b'GYOHFaVFp3o'] labels : [3]
Example gemaakt! Video-id: [b'dnFBtHwQI1I'] labels : [8]
Example gemaakt! Video-id: [

Example gemaakt! Video-id: [b'bUMYneCPJ9c'] labels : [4]
Example gemaakt! Video-id: [b'jCzviBJxnqU'] labels : [1]
Example gemaakt! Video-id: [b'-kRxowDMwdQ'] labels : [0]
Example gemaakt! Video-id: [b'8SYLYWR47EE'] labels : [0]
Example gemaakt! Video-id: [b'IPP2G8yOYEQ'] labels : [0]
Example gemaakt! Video-id: [b'nJKkJuff4cc'] labels : [2]
Example gemaakt! Video-id: [b'bmv356vIUAc'] labels : [6]
Example gemaakt! Video-id: [b'ncmbVqH7rKE'] labels : [10]
Example gemaakt! Video-id: [b'3vz2Pla29cQ'] labels : [6]
Example gemaakt! Video-id: [b'-Q9WotFs1MI'] labels : [2]
Example gemaakt! Video-id: [b'-cWrzrfVdMQ'] labels : [3]
Example gemaakt! Video-id: [b'-cQV8iXK3pQ'] labels : [3]
Example gemaakt! Video-id: [b'QwuuTg49fuE'] labels : [0]
Example gemaakt! Video-id: [b'UlVvWsYNy44'] labels : [0]
Example gemaakt! Video-id: [b'2YZ4psKel4k'] labels : [10]
Example gemaakt! Video-id: [b'2Yy-PiC089M'] labels : [7]
Example gemaakt! Video-id: [b'unUq1Sk0D_8'] labels : [4]
Example gemaakt! Video-id: [b

Example gemaakt! Video-id: [b'wxw0sHBvkR0'] labels : [8]
Example gemaakt! Video-id: [b'IzQWEa7xkpc'] labels : [2]
Example gemaakt! Video-id: [b'EwAUvy_WqaI'] labels : [8]
Example gemaakt! Video-id: [b'LWOjJShT0uA'] labels : [6]
Example gemaakt! Video-id: [b'X2Urv2KkQqU'] labels : [10]
Example gemaakt! Video-id: [b'mxgSJtIvyQg'] labels : [0]
Example gemaakt! Video-id: [b'8zn9L4SRjCs'] labels : [8]
Example gemaakt! Video-id: [b'8z2yFtX66C4'] labels : [8]
Example gemaakt! Video-id: [b'6FQ1nQ0FvgQ'] labels : [5]
Example gemaakt! Video-id: [b'I_DXFVEn4w8'] labels : [6]
Example gemaakt! Video-id: [b'Ik1exDY-k_0'] labels : [2]
Example gemaakt! Video-id: [b'1nYZnNEKI6o'] labels : [10]
Example gemaakt! Video-id: [b'Rblq9XQG6TE'] labels : [0]
Example gemaakt! Video-id: [b'apiwldD6mRo'] labels : [6]
Example gemaakt! Video-id: [b'I9Y1e1lAH_Y'] labels : [9]
Example gemaakt! Video-id: [b'Xazsdjxx8lM'] labels : [3]
Example gemaakt! Video-id: [b'5FvDc0UBAZo'] labels : [0]
Example gemaakt! Video-id: [b

Example gemaakt! Video-id: [b'ry4PXhwNgI4'] labels : [10]
Example gemaakt! Video-id: [b'l7mSUe1bxyE'] labels : [3]
Example gemaakt! Video-id: [b'po-tnKZAzdg'] labels : [0]
Example gemaakt! Video-id: [b'iXPOBBezMzY'] labels : [5]
Example gemaakt! Video-id: [b'iXgQI3dth_8'] labels : [1]
Example gemaakt! Video-id: [b'iXCg-9B_K_8'] labels : [3]
Example gemaakt! Video-id: [b'TBBXRQHKFcE'] labels : [9]
Example gemaakt! Video-id: [b'0EkXQKzNjkg'] labels : [0]
Example gemaakt! Video-id: [b'-3IGxVTJvgI'] labels : [7]
Example gemaakt! Video-id: [b'y6jEec1e6Gw'] labels : [1]
Example gemaakt! Video-id: [b'M_F44wV4Ddo'] labels : [2]
Example gemaakt! Video-id: [b'G8l2fpUPWro'] labels : [3]
Example gemaakt! Video-id: [b'6R_63fbeiR0'] labels : [7]
Example gemaakt! Video-id: [b'JmBQqW8HFDo'] labels : [6]
Example gemaakt! Video-id: [b'qxLdv8u_Ujw'] labels : [0]
Example gemaakt! Video-id: [b'11-d3rWQxZY'] labels : [10]
Example gemaakt! Video-id: [b'DsFvp-0mEZA'] labels : [10]
Example gemaakt! Video-id: [

In [ ]:
## Maak nieuwe tf-records van unbal_train
# LETOP deze is groot en duurt lang, voor tests niet nodig

files = gfile.Glob(unbal_train_data_pattern)

for file in files:
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'U')

In [17]:
## Maak nieuwe tf-records van eval

files = gfile.Glob(eval_data_pattern)

for file in files:
    
    ## Read a file and rewrite it
    readWriteTfRecord(file, 'E')

Example gemaakt! Video-id: [b'VmoG9nSADn8'] labels : [4]
Example gemaakt! Video-id: [b'2-Yz378XB2Q'] labels : [0]
Example gemaakt! Video-id: [b'ycNUMWqASrw'] labels : [9]
Example gemaakt! Video-id: [b'ycqDMKTrvLY'] labels : [0, 7]
Example gemaakt! Video-id: [b'U9o30JY6aYc'] labels : [3]
Example gemaakt! Video-id: [b'NY1zBHjxxc4'] labels : [6]
Example gemaakt! Video-id: [b'UDlc1slA8PA'] labels : [4]
Example gemaakt! Video-id: [b'3KQxT20mY-k'] labels : [6]
Example gemaakt! Video-id: [b'SRjsxeOWFnw'] labels : [10, 0]
Example gemaakt! Video-id: [b'0zDEut-yfis'] labels : [2]
Example gemaakt! Video-id: [b'0zgffMTkGtU'] labels : [8]
Example gemaakt! Video-id: [b'lXUcalwkWCo'] labels : [1]
Example gemaakt! Video-id: [b'J5om8xfMYYw'] labels : [0]
Example gemaakt! Video-id: [b'UXjhevjf9b0'] labels : [4]
Example gemaakt! Video-id: [b'eoINKEFklgM'] labels : [6]
Example gemaakt! Video-id: [b'w2bAs3lug3c'] labels : [6]
Example gemaakt! Video-id: [b'WT_wvvEvkw4'] labels : [9]
Example gemaakt! Video-i

Example gemaakt! Video-id: [b'6Y8bKS6KLeE'] labels : [0]
Example gemaakt! Video-id: [b'302w8uJn_zE'] labels : [0]
Example gemaakt! Video-id: [b'kJ1AWz_mhmE'] labels : [0]
Example gemaakt! Video-id: [b'154WG9Gv1I4'] labels : [9]
Example gemaakt! Video-id: [b'h__4MI_GgpM'] labels : [3]
Example gemaakt! Video-id: [b'NcQyU0QWPWo'] labels : [7]
Example gemaakt! Video-id: [b'ZfkO1HlI0zM'] labels : [0]
Example gemaakt! Video-id: [b'ELvsokKcydM'] labels : [6]
Example gemaakt! Video-id: [b'ELmcymPeitI'] labels : [0]
Example gemaakt! Video-id: [b'MmOWkYClDUY'] labels : [7]
Example gemaakt! Video-id: [b'E_6XYa_WO8I'] labels : [2]
Example gemaakt! Video-id: [b'Smk2zL7_L14'] labels : [7]
Example gemaakt! Video-id: [b'_uswoJHYy14'] labels : [0]
Example gemaakt! Video-id: [b'M4GU1gE8BYo'] labels : [0]
Example gemaakt! Video-id: [b'lJTpC9bjne0'] labels : [10]
Example gemaakt! Video-id: [b'2W4dw8XuetE'] labels : [10]
Example gemaakt! Video-id: [b'vauFW9Nys8Q'] labels : [2]
Example gemaakt! Video-id: [b

Example gemaakt! Video-id: [b'I1N1eIQubFc'] labels : [8]
Example gemaakt! Video-id: [b'ikmE_kRvDAc'] labels : [0]
Example gemaakt! Video-id: [b'XAgX4QEV9Ts'] labels : [6]
Example gemaakt! Video-id: [b'BI4quuGz_zk'] labels : [0]
Example gemaakt! Video-id: [b'BI_Llv8hnC0'] labels : [10]
Example gemaakt! Video-id: [b'0w7pCxwXzVc'] labels : [10]
Example gemaakt! Video-id: [b'B98yOZSj2GE'] labels : [0]
Example gemaakt! Video-id: [b'9kyrVsiNZOU'] labels : [3]
Example gemaakt! Video-id: [b'EzXThJXRdbI'] labels : [3]
Example gemaakt! Video-id: [b'AkUDv7JexjQ'] labels : [10, 0]
Example gemaakt! Video-id: [b'sbpW3Z87Nbc'] labels : [6]
Example gemaakt! Video-id: [b'RvyCH9e32mw'] labels : [9]
Example gemaakt! Video-id: [b'tFLGKmOa0dc'] labels : [6]
Example gemaakt! Video-id: [b'J2nDnAerah0'] labels : [4]
Example gemaakt! Video-id: [b'AQXT9gOjT4s'] labels : [7]
Example gemaakt! Video-id: [b'd_Hdkt1_ivo'] labels : [10, 0]
Example gemaakt! Video-id: [b'c-qYpm6h0Ew'] labels : [0]
Example gemaakt! Vide

Example gemaakt! Video-id: [b'eP_jVxMS8AE'] labels : [9]
Example gemaakt! Video-id: [b'SGFYFPs3Fic'] labels : [4]
Example gemaakt! Video-id: [b'bVR288WbDXc'] labels : [0]
Example gemaakt! Video-id: [b'bVPLdRReUZ8'] labels : [3]
Example gemaakt! Video-id: [b'KnjgYUAlbdo'] labels : [7]
Example gemaakt! Video-id: [b'SoOIUzwU8Gs'] labels : [9]
Example gemaakt! Video-id: [b'SoKwtehYi9E'] labels : [8]
Example gemaakt! Video-id: [b'Q6ThTeKdgR0'] labels : [0]
Example gemaakt! Video-id: [b'RMPyriqNS-4'] labels : [6]
Example gemaakt! Video-id: [b'dwlXkSGFgRI'] labels : [2]
Example gemaakt! Video-id: [b'CkutJYIfghs'] labels : [10]
Example gemaakt! Video-id: [b'_mr2Rpuqlyw'] labels : [2]
Example gemaakt! Video-id: [b'pNMBIqvbyB4'] labels : [0]
Example gemaakt! Video-id: [b'G3GR-m8Q3Zk'] labels : [3]
Example gemaakt! Video-id: [b'F-Ly8CI7-Pc'] labels : [3]
Example gemaakt! Video-id: [b'0TV9zvfwFhs'] labels : [5]
Example gemaakt! Video-id: [b'KqsNyXIEIwg'] labels : [0]
Example gemaakt! Video-id: [b'

Example gemaakt! Video-id: [b'uIm46Vckbhg'] labels : [8]
Example gemaakt! Video-id: [b'L-syzwlD05s'] labels : [0]
Example gemaakt! Video-id: [b'_w8u8H6TP_o'] labels : [0]
Example gemaakt! Video-id: [b'ztfXxgXqHGM'] labels : [3]
Example gemaakt! Video-id: [b'nxeuS5dkRPk'] labels : [8]
Example gemaakt! Video-id: [b'WLX3Db60418'] labels : [0]
Example gemaakt! Video-id: [b'WLPj0LIokkM'] labels : [1]
Example gemaakt! Video-id: [b'R5mK5xASBXM'] labels : [8]
Example gemaakt! Video-id: [b'wlCx3oOCxAk'] labels : [0]
Example gemaakt! Video-id: [b'e9EUmQBO498'] labels : [0]
Example gemaakt! Video-id: [b'fvWS01GjKIw'] labels : [10]
Example gemaakt! Video-id: [b'q1gQRZ3Hi1I'] labels : [0]
Example gemaakt! Video-id: [b'_IRW0owPIOA'] labels : [3]
Example gemaakt! Video-id: [b'BZRZcFA9Dr4'] labels : [6]
Example gemaakt! Video-id: [b'6mdJY-GVdvs'] labels : [5]
Example gemaakt! Video-id: [b'Bt_kR7u6mM4'] labels : [9]
Example gemaakt! Video-id: [b'_gesDZ5_rAg'] labels : [3]
Example gemaakt! Video-id: [b'

Example gemaakt! Video-id: [b'dcjbs7R1yL8'] labels : [6]
Example gemaakt! Video-id: [b'CZQ1bedI5Wo'] labels : [1]
Example gemaakt! Video-id: [b'CZgx_6XaEkg'] labels : [0]
Example gemaakt! Video-id: [b'CZlQbIE6FXQ'] labels : [10]
Example gemaakt! Video-id: [b'5hSnO76tkIM'] labels : [10]
Example gemaakt! Video-id: [b'gKZrAlTJURs'] labels : [6]
Example gemaakt! Video-id: [b'ovgelLpt148'] labels : [3]
Example gemaakt! Video-id: [b'QdyhX0sGGPE'] labels : [7]
Example gemaakt! Video-id: [b'bKzyLIonf0c'] labels : [0]
Example gemaakt! Video-id: [b'IXM05dPfKmo'] labels : [10]
Example gemaakt! Video-id: [b'nMaSkwx6cHE'] labels : [0]
Example gemaakt! Video-id: [b'DJCkQUdA9QQ'] labels : [2]
Example gemaakt! Video-id: [b'jJkCnY3keZk'] labels : [0]
Example gemaakt! Video-id: [b'ou0oVoBnLaM'] labels : [7]
Example gemaakt! Video-id: [b'O7-ggV4SQRs'] labels : [4]
Example gemaakt! Video-id: [b'g1D25N2Q0fI'] labels : [3]
Example gemaakt! Video-id: [b'z_8yGVO1qws'] labels : [0]
Example gemaakt! Video-id: [

Example gemaakt! Video-id: [b'GNN6PhC6F8Q'] labels : [8]
Example gemaakt! Video-id: [b'GNYKzDo4w7w'] labels : [3]
Example gemaakt! Video-id: [b'6SVS8wrJ1yY'] labels : [3]
Example gemaakt! Video-id: [b'6S8ewP_bduQ'] labels : [2]
Example gemaakt! Video-id: [b'D_GfvnDbRJk'] labels : [10]
Example gemaakt! Video-id: [b'wH45h-0Kld8'] labels : [8]
Example gemaakt! Video-id: [b'DB38NRSHw9A'] labels : [5]
Example gemaakt! Video-id: [b'gxVhAVNjSU0'] labels : [0]
Example gemaakt! Video-id: [b'gx6wJrRxq-4'] labels : [10]
Example gemaakt! Video-id: [b'WatvT8A8iug'] labels : [0]
Example gemaakt! Video-id: [b'gvuoL6ugzvw'] labels : [1]
Example gemaakt! Video-id: [b'JLzD44Im1Ec'] labels : [0]
Example gemaakt! Video-id: [b'cGUhG5PZp0A'] labels : [5]
Example gemaakt! Video-id: [b'5S9W9-nTgLE'] labels : [10]
Example gemaakt! Video-id: [b'mEslZvugQCc'] labels : [5]
Example gemaakt! Video-id: [b'YNfqO5oHGaA'] labels : [3]
Example gemaakt! Video-id: [b'UyCfhj3LXVk'] labels : [6]
Example gemaakt! Video-id: [

Example gemaakt! Video-id: [b'lmaIgor5XTg'] labels : [0]
Example gemaakt! Video-id: [b'M7rGKqfZh6k'] labels : [3]
Example gemaakt! Video-id: [b'Do5DfkHBhtg'] labels : [1]
Example gemaakt! Video-id: [b'VpikuLP_9qQ'] labels : [7]
Example gemaakt! Video-id: [b'Vp4DzRFSeq8'] labels : [7]
Example gemaakt! Video-id: [b'MSIBrBs56wg'] labels : [6]
Example gemaakt! Video-id: [b'Vsz6dpRCwSo'] labels : [1]
Example gemaakt! Video-id: [b'3OlQmVVOzbQ'] labels : [10]
Example gemaakt! Video-id: [b'flKCxPX1EHo'] labels : [9]
Example gemaakt! Video-id: [b'TQsjIE2VHqw'] labels : [1]
Example gemaakt! Video-id: [b't0txEC0Rhdg'] labels : [4]
Example gemaakt! Video-id: [b'MuMDetHdBTc'] labels : [7]
Example gemaakt! Video-id: [b'UYBuKiXo92s'] labels : [6]
Example gemaakt! Video-id: [b'wVMkSyxQ6IM'] labels : [10, 0]
Example gemaakt! Video-id: [b'8sd513xQzV4'] labels : [10]
Example gemaakt! Video-id: [b'xMClk12ouB8'] labels : [0]
Example gemaakt! Video-id: [b'2Jpg_KvJWL0'] labels : [0]
Example gemaakt! Video-id

In [19]:
## Print voor alle labels het aantal voorkomens in de nieuwe tf-records

sum_occur = 0
cnt_labels = Counter(all_labels)

for i in cnt_labels.most_common(100):
    sum_occur = sum_occur + i[1]
    print(label_proportions.loc[label_proportions['index']==i[0]]['display_name'].values[0], ' : ', str(i[1]))

Car, boat, motorcyle, truck  :  819
Gunshot, gunfire  :  526
Wind noise (microphone)  :  441
Cattle  :  360
Male singing  :  182
Chop  :  180
Helicopter  :  180
Chainsaw  :  180
Shout  :  180
Radio  :  180
Silence  :  178


In [20]:
## Tellen van lijsten in lijsten is lastig, vandaar dat ik er strings van maak: [0,4] wordt ["0_4"]
str_combis = []

for combi in label_combinations:
    str_combi = [str(i) for i in combi]
    str_combi = "_".join(str_combi)
    str_combis.append(str_combi)

In [21]:
## Houd individuele voorkomens bij (dus aantal keer dat label voorkomt zonder andere labels)
ind_occur = [0 for x in range(label_proportions.shape[0])] 

## Geef meest voorkomende combinaties
cnt_label_combinations = Counter(str_combis)
for label_combination in cnt_label_combinations.most_common(20):
    label_names = []
    
    ## Maak van string weer een lijst (zie vorige cel)
    label_list = label_combination[0].split("_")
    
    ## Code die alleen individuele labels telt
    if(len(label_list)==1):
        ind_occur[int(label_list[0])] = label_combination[1]
    
    for i in label_list:
        label_names.append(str(label_proportions.loc[label_proportions['index']==i]['display_name']))
    label_names = ', '.join(label_names)
    print(label_names,': ', label_combination[1])

Series([], Name: display_name, dtype: object) :  757
Series([], Name: display_name, dtype: object) :  525
Series([], Name: display_name, dtype: object) :  388
Series([], Name: display_name, dtype: object) :  360
Series([], Name: display_name, dtype: object) :  182
Series([], Name: display_name, dtype: object) :  180
Series([], Name: display_name, dtype: object) :  180
Series([], Name: display_name, dtype: object) :  178
Series([], Name: display_name, dtype: object) :  178
Series([], Name: display_name, dtype: object) :  175
Series([], Name: display_name, dtype: object) :  171
Series([], Name: display_name, dtype: object), Series([], Name: display_name, dtype: object) :  51
Series([], Name: display_name, dtype: object), Series([], Name: display_name, dtype: object) :  7
Series([], Name: display_name, dtype: object), Series([], Name: display_name, dtype: object) :  2
Series([], Name: display_name, dtype: object), Series([], Name: display_name, dtype: object) :  2
Series([], Name: display

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [22]:
print("sum occurences: " + str(sum_occur))
print("ind occ: ")
print(ind_occur)

sum occurences: 3406
ind occ: 
[757, 171, 180, 525, 175, 182, 360, 178, 180, 178, 388, 0, 0, 0, 0]


In [ ]:
## UP-/DOWN-SAMPLEN

## Alle code hierna is alleen om uit te rekenen wat de proporties zijn
## Het werkt als volgt:
## Van het totale aantal voorkomens van een label (bv 11.000), wordt het gewenste afgetrokken:
## bv 11.000-2.000 = 9.000
## Dit aantal examples moet geskipt worden, maar dan alleen als het label alleen voorkomt
## Daarom wordt de target proportie van het totale individuutjes (target_ind) als volgt berekend:
## Stel ind_occ (aantal keer alleen voorkomend) is 10.000, waarvan we er 9.000 niet willen hebben.
## (10.000-9.000)/10.000 = 0.1 is dan de kans dat een example wordt geschreven.

In [ ]:
## Code die kolom toevoegt aan label-csv (proportion) en deze in de forloop vult

## Aantal gewenste samples per label: 2500
target = 2500

## Aantal voorkomens en aantal voorkomens alleen
label_proportions["total"] = 0
label_proportions["ind_total"] = 0
label_proportions["proportion"] = 0
label_proportions["target_ind"] = 0

for i in range(mid_to_label_new.shape[0]):
    label_proportions.loc[i,"total"] = cnt_labels[i]
    label_proportions.loc[i,"ind_total"] = ind_occur[i]   
    label_proportions.loc[i,"proportion"] = cnt_labels[i]/sum_occur
    label_proportions.loc[i,"target_ind"] = (ind_occur[i]-(cnt_labels[i]-target))/ind_occur[i]

In [ ]:
## Schrijf bestandje weg
label_proportions.to_csv(name_label_prop_file, sep = ";")

In [ ]:
label_proportions

In [ ]:
#### Lees tf-records en iterate over de examples#### Lee 
#### Voor elk example wordt de video-id gepakt en de bijbehorende lijst m-id's gezocht (in balanced_segment.csv)

## 1 tf-record heeft meerdere examples
## Deze functie checkt voor elk example of het een label heeft uit onze labelset

## Input: filename van tf-record (pad) en prefix (vanwege dubbele filenames)
## Result: schrijven van nieuw tf-record

## Geef prefix mee (om dubbele namen te voorkomen)
def readTfRecord(tfrecords_filename):
    
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        print(example)
        ## Get labels and find for each label the m-code in class_label_indices_old
        labels = example.context.feature['labels'].int64_list.value
       
        #if(labels[0]!=11):
        #    print(labels)
        break;

In [ ]:
file = 'tfrecords/train_jungle_new/gun_train.tfrecord'

In [ ]:
readTfRecord(file)